In [66]:
import pandas as pd
import numpy as np
import json
import re 
import sys
from scipy.sparse import csr_matrix 
import itertools
import cv2
from PIL import Image

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from skimage import io
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [67]:
%matplotlib inline

In [68]:
#another useful command to make data exploration easier
# NOTE: if you are using a massive dataset, this could slow down your code. 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [90]:
df = pd.read_csv('D:\\Music-recommendation-based-on-facial-emotion-recognition-main (1)\\Music-recommendation-based-on-facial-emotion-recognition-main\\Datasets\\kaggleMusicMoodFinal.csv')

In [91]:
spotify_df = df.head(100).copy()

In [92]:
spotify_df.shape

(100, 27)

In [93]:
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values

In [94]:
ohe_cols = 'popularity'

In [95]:
# create 5 point buckets for popularity 
spotify_df['bucket_popularity'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [96]:
spotify_df['consolidates_genre_lists_upd'] = spotify_df['consolidates_genre_lists'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [97]:
spotify_df.head()

,Unnamed: 0.1,Unnamed: 0,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,Mood,bucket_popularity,consolidates_genre_lists_upd
0,0,0,0.177,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,0.000001,10,0.0649,-16.478,1,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,['조정현'],[],['조정현'],조정현그아픔까지사랑한거야,['classic_korean_pop'],Sad,6,[classic_korean_pop]
1,1,1,0.352,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,11,0.0568,-9.103,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,['黑豹'],[],['黑豹'],黑豹DON'T BREAK MY HEART,"['chinese_indie', 'chinese_indie_rock']",Sad,7,"[chinese_indie, chinese_indie_rock]"
2,2,2,0.458,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,5,0.2650,-24.571,1,藝旦調,23,1963-05-28,0.0443,85.936,['黃國隆'],[],['黃國隆'],黃國隆藝旦調,[],Sad,4,[]
3,3,3,0.796,1963,0.852,"['黃國隆', '王秋玉']",0.711,145720,0.1110,0,5xFXTvnEe03SyvFpo6pEaE,0.000000,2,0.0695,-20.741,0,草螟弄雞公,23,1963-05-28,0.0697,124.273,"['黃國隆', '王秋玉']",[],"['黃國隆', '王秋玉']",黃國隆草螟弄雞公,[],Sad,4,[]
4,4,4,0.704,1963,0.771,['黃國隆'],0.610,208760,0.1750,0,6Pqs2suXEqCGx7Lxg5dlrB,0.000000,7,0.0309,-20.232,1,思想起,23,1963-05-28,0.0419,124.662,['黃國隆'],[],['黃國隆'],黃國隆思想起,[],Sad,4,[]


In [134]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()
client_id = '3a98f83d33c547668bcd83dbd32eb699'
client_secret = '93f3ad98a52545d0884323e1ad45344a'    

In [135]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [136]:
token = util.prompt_for_user_token(scope, client_id=client_id, client_secret=client_secret, redirect_uri='http://localhost:8400/')

In [137]:
sp = spotipy.Spotify(auth=token)

In [138]:
#gather playlist names and images. 
#images aren't going to be used until I start building a UI
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [139]:
id_name

{'Happy': '4mluGjfzgdHISqxcgmAt03',
 'Telgu': '6ZSXLok2nhRaepBTFKDmRO',
 'Punjabi': '5qesxpIzXb6KztJTwAz6q6',
 'Morning': '63t2eyGKeUEYA21JbJeOOW',
 'Yo': '00VKfDv9RHDao3gUgVIU4n',
 'Love': '7DPXdbezN1cwYPSRbNaVAy'}

In [149]:
def ChooseDataset(x):
    if x == "Disgust":
        return spotify_df[spotify_df['Mood'].isin(['Energetic', 'Happy', 'Calm'])]
    if x == "Angry":
        return spotify_df[spotify_df['Mood'].isin(['Energetic', 'Calm'])]
    if x == "Fear":
        return spotify_df[spotify_df['Mood'].isin(['Happy', 'Calm'])]
    if x == "Happy":
        return spotify_df[spotify_df['Mood'].isin(['Sad', 'Happy', 'Calm'])]
    if x == "Sad":
        return spotify_df[spotify_df['Mood'].isin(['Sad', 'Happy', 'Calm'])]
    if x == "Surprise":
        return spotify_df[spotify_df['Mood'].isin(['Energetic', 'Happy', 'Sad'])]
    return spotify_df

In [114]:
from tensorflow.keras.models import load_model

loaded_model = load_model('D:\\Music-recommendation-based-on-facial-emotion-recognition-main (1)\\Music-recommendation-based-on-facial-emotion-recognition-main\\new_img_data\\model.keras', compile = True)

In [115]:
def moodNamePrintFromLabel(n):
    if n == 0: result = "Angry"
    elif n == 1: result = "Disgust"
    elif n == 2: result = "Fear"
    elif n == 3: result = "Happy"
    elif n == 4: result = "Sad"
    elif n == 5: result = "Surprise"
    elif n == 6: result = "Neutral"
        
    return result
face_casecade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def imageToMoodDetection(img_path):

    img = cv2.imread(img_path)

    Gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_casecade.detectMultiScale(Gray_img, 1.3, 3)

    for (x,y,w,h) in faces:
      pxl_lst = []
      for i in range(y, y+h):
        lst = []
        for j in range(x, x+w):
          lst.append(Gray_img[i][j])
        pxl_lst.append(lst)
      single_face = np.array(pxl_lst)
      single_face = np.reshape(single_face,(h,w,))

      resized_img = cv2.resize(single_face, (48,48), interpolation = cv2.INTER_AREA)
      
      resized_img = np.reshape(resized_img,(1,48,48,1))/255.0

      #passing to model
      result = np.argmax(loaded_model.predict(resized_img), axis=-1)
      if result is not None:
        print(moodNamePrintFromLabel(result))

In [165]:
def Recommend_Music(emotion, df, features, nonplaylist_features):
    """ 
    Recommend songs based on the user's emotion.

    Parameters: 
        emotion (str): The user's most recent emotion.
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        recommended_songs_df: Recommended songs based on the emotion
    """
    
    # Filter songs based on the emotion (you may need to customize this based on your dataset)
    emotion_filtered_df = df[df['emotion_column'] == emotion]
    
    # Calculate features for the emotion-filtered songs
    emotion_filtered_features = Feature_Set_Using_TDiDF(emotion_filtered_df, float_cols=float_cols)
    
    # Combine the emotion-specific features with the overall features
    combined_features = features + emotion_filtered_features.mean(axis=0)
    
    # Recommended Songs
    recommended_songs_df = Recommend_Playlist(df, combined_features, nonplaylist_features)
    
    return recommended_songs_df.head(10)

In [179]:
def One_Hot_Encode_Prep(df, column, new_name):
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop=True, inplace=True)
    return tf_df

# Function to build entire feature set
def Feature_Set_Using_TDiDF(df, float_cols):
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['consolidates_genre_lists_upd'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray(), columns=['genre' + "|" + i for i in tfidf.get_feature_names_out()])

    year_ohe = One_Hot_Encode_Prep(df, 'year', 'year') * 0.5
    popularity_ohe = One_Hot_Encode_Prep(df, 'bucket_popularity', 'pop') * 0.15

    floats = df[float_cols].reset_index(drop=True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns=floats.columns) * 0.2

    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis=1)
    final['id'] = df['id'].values

    return final


# Function to pull songs from a specific playlist
def Get_Spotify_Playlist_DataFrame(playlist_name, id_dic, df):
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        playlist.loc[ix, 'artist'] = i['track']['album']['artists'][0]['name']
        playlist.loc[ix, 'album'] = i['track']['album']['name']
        playlist.loc[ix, 'id'] = i['track']['id']
        playlist.loc[ix, 'track_name'] = i['track']['name']
        playlist.loc[ix, 'added_at'] = i['added_at']

    playlist = pd.merge(playlist, df, how='inner', on='id')
    playlist['date_added'] = pd.to_datetime(playlist['added_at'])
    playlist.drop(columns='added_at', inplace=True)
    return playlist

# Modified function to generate playlist features
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    playlist_features_in_dataframe = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    playlist_features_in_dataframe = playlist_features_in_dataframe.merge(playlist_df[['id', 'date_added']], on='id',
                                                                          how='inner')

    # Check if the playlist is not empty
    if not playlist_features_in_dataframe.empty:
        playlist_features_not_in_dataframe = complete_feature_set[
            ~complete_feature_set['id'].isin(playlist_df['id'].values)]

        playlist_feature_set = playlist_features_in_dataframe.sort_values('date_added', ascending=False)

        most_recent_date = playlist_feature_set.iloc[0, -1]

        for ix, row in playlist_feature_set.iterrows():
            playlist_feature_set.loc[ix, 'months_from_recent'] = int(
                (most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)

        playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))

        playlist_feature_set_weighted = playlist_feature_set.copy()
        playlist_feature_set_weighted.update(
            playlist_feature_set_weighted.iloc[:, :-4].mul(playlist_feature_set_weighted.weight, 0))
        playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4].mean(axis=0)

        return playlist_feature_set_weighted_final, playlist_features_not_in_dataframe
    else:
        # Handle the case where the playlist is empty
        return None, None


# Function to recommend songs
def Recommend_Playlist(spotify_df, features_in_the_playlist, features_not_in_the_playlist):
    # Recommendation logic here
    # ...

    # Dummy data, replace with your recommendation logic
    recommended_songs = spotify_df.sample(10)

    return recommended_songs

# Function to visualize recommended songs
def visualize_songs(top40_recommendation):
    # Visualization logic here
    # ...

    print(top40_recommendation)

# Main function for recommendation
def Recommend_Top40(emotions, loaded_model, spotify_df):
    # Choose the dataset based on the emotion
    emotion_dict = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
    emotion = emotion_dict[emotions[-1]]  # Use the most recent emotion

    O_df = ChooseDataset(emotion)

    # Feature Engineering from the main dataframe
    float_cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'tempo',
                  'valence']
    complete_feature_set = Feature_Set_Using_TDiDF(O_df, float_cols)

    # Collecting the Spotify user playlist dataframe
    one_playlist_from_spotify = Get_Spotify_Playlist_DataFrame(emotion, id_name, O_df)

    # Linear vector for recommendation
    features_in_the_playlist, features_not_in_the_playlist = generate_playlist_feature(complete_feature_set,
                                                                                         one_playlist_from_spotify, 1.09)

    # Recommended Songs stored here
    top40_recommendation = Recommend_Playlist(spotify_df, features_in_the_playlist, features_not_in_the_playlist)

    # Visualize recommended songs
    visualize_songs(top40_recommendation)

In [180]:
def moodNamePrintFromLabel(n):
    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    return emotions[n]

# Function to capture video from webcam and perform emotion detection
def capture_and_detect_emotion(model):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    video_cap = cv2.VideoCapture(0)  # Open the webcam

    emotions = []
    start_time = cv2.getTickCount()

    while (video_cap.isOpened()):
        ret, frame = video_cap.read()

        if not ret:
            print("Failed to capture frame")
            break

        gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)

        for (x, y, w, h) in faces:
            pxl_lst = []
            for i in range(y, y + h):
                lst = []
                for j in range(x, x + w):
                    lst.append(gray_img[i][j])
                pxl_lst.append(lst)
            single_face = np.array(pxl_lst)

            resized_img = cv2.resize(single_face, (48, 48), interpolation=cv2.INTER_AREA)
            resized_img = np.reshape(resized_img, (1, 48, 48, 1)) / 255.0

            # Get the result from the model
            result = np.argmax(model.predict(resized_img), axis=-1)

            # Draw a rectangle around the face and display the mood label
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, moodNamePrintFromLabel(result[0]), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                        (255, 0, 0), 2)

            emotions.append(result[0])

        # Display the frame with mood information
        cv2.imshow('Webcam Mood Detection', frame)

        current_time = cv2.getTickCount()
        elapsed_time = (current_time - start_time) / cv2.getTickFrequency()

        if elapsed_time >= 5:
            break  # Break after 5 seconds

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the window
    video_cap.release()
    cv2.destroyAllWindows()

    return emotions

In [181]:
# Capture emotions from webcam
emotions = capture_and_detect_emotion(loaded_model)

# Recommend songs based on the most recent emotion
Recommend_Top40(emotions, loaded_model, spotify_df)

1/1 [==============================] - 0s 31ms/step
    Unnamed: 0.1  Unnamed: 0  valence  year  acousticness  \
22            22          22   0.0718  1966       0.94100   
70            70          70   0.5000  1977       0.64400   
23            23          23   0.4370  1963       0.72600   
6              6           6   0.8190  1963       0.69100   
99            99          99   0.6680  1982       0.19900   
50            50          50   0.3750  1963       0.95800   
17            17          17   0.6350  1963       0.91600   
77            77          77   0.7570  1942       0.80200   
28            28          28   0.1150  1963       0.97300   
67            67          67   0.7960  2018       0.00655   

                                              artists  danceability  \
22                                           ['霜華雪華']         0.255   
70                                            ['許冠英']         0.490   
23                                           ['霜華雪華']         0

In [88]:
Image_path = 'D:\\1GB-img\\Happy\\21.png'

# Recommended songs for 'EDM' Playlist for Neutral Mood

In [113]:
Recommend_Top40(imageToMoodDetection(Image_path))

1/1 [==============================] - 0s 31ms/step
Neutral
1/1 [==============================] - 0s 30ms/step
Angry
1/1 [==============================] - 0s 31ms/step
Angry


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required by check_pairwise_arrays.